In [3]:
import os

In [31]:
# Read reviews from training file
reviews = open('data/train.txt', 'r').read().split('\n')
reviews = [line.split('\t') for line in reviews]

# Validate reviews strings
for r in reviews:
    if len(r) is not 2:
        print('Rejected invalid input: ', str(r))
        reviews.remove(r)

rw_pos = [r[0] for r in reviews if r[1] == '1']
rw_neg = [r[0] for r in reviews if r[1] == '0']

Rejected invalid input:  ['']


In [33]:
rw_pos

['Wow... Loved this place.',
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.',
 'The selection on the menu was great and so were the prices.',
 'The fries were great too.',
 'A great touch.',
 'Service was very prompt.',
 'I tried the Cape Cod ravoli, chicken,with cranberry...mmmm!',
 'Highly recommended.',
 'The food, amazing.',
 'Service is also cute.',
 'I could care less... The interior is just beautiful.',
 'So they performed.',
 "That's right....the red velvet cake.....ohhh this stuff is so good.",
 'This hole in the wall has great Mexican street tacos, and friendly staff.',
 'Also there are combos like a burger, fries, and beer for 23 which is a decent deal.',
 'I found this place by accident and I could not be happier.',
 'Overall, I like this place a lot.',
 'The only redeeming quality of the restaurant was that it was very inexpensive.',
 'Ample portions and good prices.',
 'My first visit to Hiro was a delight!',
 'The shrimp tender 